# Step 1: Read & Process Transcript Files
We first read the transcript text files, grouping together the files in the "Free" group as one set, and the "D-C" group as another set.

To avoid processing errors, we first remove all non-UTF characters in the text (such as distinct opening and closing quotes)
and replace them with corresponding UTF characters. These transcripts also come with the transcriber's observations, such as [laughter], or [silent]. We remove all such observations so that we are only looking at team members' speech.

Finally we remove all stop words

In [1]:
import re
# Locations of input files. These files have been anonymized, and the files themselves will not be available
# in this folder, to preserve the privacy of the teams.
textPathFree = "../../../private/brainstorming_viz_data/free_group/"
textPathDC = "../../../private/brainstorming_viz_data/d-c_group/"

# Teamwise transcript (of divergent processes) of the "Free Brainstorming" group
fileNamesFreeDivergent = ['teamA_part1_divergent.txt', 'teamB_part1_divergent.txt', 'teamC_part1_divergent.txt',
                          'teamD_part1_divergent.txt', 'teamE_part1_divergent.txt', 'teamF_part1_divergent.txt',
                          'teamG_part1_divergent.txt', 'teamH_part1_divergent.txt', 'teamI_part1_divergent.txt']

# Teamwise transcript (of divergent processes) of the "Divergent-Convergent (D-C) Brainstorming" group
fileNamesDCDivergent = ['teamJ_part1_divergent.txt', 'teamJ_part3_divergent.txt', 'teamK_part1_divergent.txt',
                        'teamK_part3_divergent.txt', 'teamL_part1_divergent.txt', 'teamL_part3_divergent.txt', 
                        'teamM_part1_divergent.txt', 'teamM_part2_3_convergent_divergent.txt', 'teamN_part1_divergent.txt',
                        'teamN_part3_divergent.txt', 'teamO_part1_divergent.txt', 'teamO_part2_3_conv_div.txt',
                        'teamP_part1_divergent.txt', 'teamP_part3_divergent.txt', 'teamQ_part1_divergent.txt',
                        'teamQ_part3_4_div_conv.txt', 'teamR_part1_divergent.txt', 'teamR_part3_divergent.txt'] 

# Teamwise transcript (of divergent processes) of the "Free Brainstorming" group
fileNamesFreeConvergent = ['teamA_part2_convergent.txt', 'teamB_part2_convergent.txt', 'teamC_part2_convergent.txt',
                           'teamD_part2_convergent.txt', 'teamE_part2_convergent.txt', 'teamF_part2_convergent.txt',
                           'teamG_part2_convergent.txt', 'teamH_part2_convergent.txt', 'teamI_part2_convergent.txt']

# Teamwise transcript (of divergent processes) of the "Divergent-Convergent (D-C) Brainstorming" group
fileNamesDCConvergent = ['teamJ_part2_convergent.txt', 'teamJ_part4_convergent.txt', 'teamK_part2_convergent.txt',
                         'teamK_part4_convergent.txt', 'teamL_part2_convergent.txt', 'teamL_part4_convergent.txt',
                         'teamM_part4_convergent.txt', 'teamN_part2_convergent.txt', 'teamN_part4_convergent.txt',
                         'teamO_part4_convergent.txt', 'teamP_part2_convergent.txt', 'teamP_part4_convergent.txt',
                         'teamQ_part2_convergent.txt', 'teamR_part2_convergent.txt', 'teamR_part4_convergent.txt']  

def openFiles(fileNamesList, filePath):
    # Given a list of file names and a path for the files, returns a
    # list of file objects
    fObjList = []
    for fileName in fileNamesList:
        fObj = open(filePath + fileName, 'r', encoding='utf8')
        fObjList.append(fObj)
    return fObjList

# Some of these files have non-UTF characters that may need to be removed.
def removeSpecials(filesList):
    # Remove some incompatible unicode characters such as distinct
    # opening and closing quotes, ellipsis etc.
    # Also finally converts all text to lowercase
    outString = ''
    for inFile in filesList:
        # print("reading file",inFile.name.split("/")[-1], "...", end="")
        temp = inFile.read()
        subsFileQuotes = re.sub(u"(\u2018|\u2019)", "'", temp)
        subsFileDQuotes = re.sub(u'(\u201c|\u201d)', '"', subsFileQuotes)
        subsFileEllipsis = re.sub(u"\u2026", "...", subsFileDQuotes)
        outString += subsFileEllipsis
        # print("done.")
    return outString.lower()

# These files have the transcriber's observations like [laughter], [silent], [cross-talk] etc.
# These may throw off the text analysis, so we remove them. These are typically within parantheses,
# so we can remove everything between these parantheses.

def removeObservations(test_str) :
    # removes all text within parantheses
    # these are usually observations by the transcriber
    # and thus should not be considered in content analysis
    # code from http://stackoverflow.com/questions/14596884/remove-text-between-and-in-python
    ret = ''
    skip1c = 0
    skip2c = 0
    squareparen = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
            squareparen += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

filesListFree = openFiles(fileNamesFreeDivergent + fileNamesFreeConvergent, textPathFree)
filesListDC = openFiles(fileNamesDCDivergent + fileNamesDCConvergent, textPathDC)

group1_raw = removeSpecials(filesListFree)
group2_raw = removeSpecials(filesListDC)


group1 = removeObservations(group1_raw)
group2 = removeObservations(group2_raw)

group1_lines = group1.split('\n')
group2_lines = group2.split('\n')

# Step 2: Computing the Latent Dirichlet Allocation
Detailed explanation to follow...


In [2]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

## Step 2.1 Computing the Document-Term Matrix

The document-term matrix looks like this:

|      | T1     | T2     | T3     | T4     | T5     |... 
|:----:|:------:|:------:|:------:|:------:|:------:|:---:
| D1   | N_d1t1 | N_d1t2 | N_d1t2 | N_d1t3 | N_d1t4 |...
| D2   | N_d2t1 | N_d2t2 | N_d2t2 | N_d2t3 | N_d2t4 |...
| D3   | N_d3t1 | N_d3t2 | N_d3t2 | N_d3t3 | N_d3t4 |...
| D4   | N_d4t1 | N_d4t2 | N_d4t2 | N_d4t3 | N_d4t4 |...
| D5   | N_d5t1 | N_d5t2 | N_d5t2 | N_d5t3 | N_d5t4 |...
| D6   | N_d6t1 | N_d6t2 | N_d6t2 | N_d6t3 | N_d6t4 |...
| ..   | ...... | ...... | ...... | ...... | ...... |...

Where N_ditj is called the **term frequency**, and is equal to the the number of occurrences of term Tj in document Di.

For our purpose, each term is a word longer than two letters, and each document is a line in the transcript. We need to focus on words that matter, so this means removing any stop words, as well as words from the transcript that we deem irrelevant. These include filler words such as "yeah", "like", etc. and non-words such as "hmm". Since this is a transcript, we find a number of time-stamped entries embedded in the text, so these need to go as well. A complete list of words to remove from the list of terms is below (excluding stop words).

Each document is represented as an M-dimensional vector where M is the number of terms computed for the corpus. Here, the vector corresponds to the row of the document-term matrix.

In [3]:
words_to_remove = ['yea', 'like', 'up', 'down', "!", "$", "'", "''", "'cause", 
                   "'d", "'em", "'ll", "'m", "'re", "'s", "'ve", ",", "yeah",
                   "-", ".", "1", "10", "12", "12.", "15", "20", "3-d", 
                   "30", "300", "352.", "36", "375", "3d", "452", "475", 
                   "475.", "90", ":", ";", "?", "``", "one", "would", "right",
                   "okay", 'could', 'know', 'make', 'something', 'yes', 'good',
                   "think", "oh", "maybe", 'kinda', 'actually', 'thing', 'things',
                   'gonna', 'wanna', 'really', 'hmm',
                   # The below set was after topic modeling
                   'cuz', 'huh', 'didn', 'wouldn', 'doesn', 'whatever', 'guys'
                  ]

complete_stop_words = stopwords.words('english') + words_to_remove

tf_vectorizer_grp1 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)

tf_vectorizer_grp2 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)
dtm_tf_grp1 = tf_vectorizer_grp1.fit_transform(group1_lines)
print("Document-Term Matrix size for Free Group:", dtm_tf_grp1.shape)
dtm_tf_grp2 = tf_vectorizer_grp2.fit_transform(group2_lines)
print("Document-Term Matrix size for D-C Group:", dtm_tf_grp2.shape)

Document-Term Matrix size for Free Group: (7641, 475)
Document-Term Matrix size for D-C Group: (3872, 285)


## 2.2 Compute the Term Frequency - Inverse Document Frequency
In order to use a scheme for weighting the document-term matrix, we use the popular **Inverse Document Frequency (IDF)** weighting measure.
The IDF is calculated as:

$IDF = log(N / n_j)$

Where $N$ is the total number of documents, and $n_j$ is the number of documents containing term $j$.
This system gives higher weightage to terms that occur in relatively fewer documents (and thus are more characteristic of the *topic* of that document)

The weighted matrix is called Term Frequency-Inverse Document Frequency, or **Tf-idf**, where each cell (term frequency) is multiplied by the weight (inverse document frequency)

In [4]:
tfidf_vectorizer_grp1 = TfidfVectorizer(**tf_vectorizer_grp1.get_params())
dtm_tfidf_grp1 = tfidf_vectorizer_grp1.fit_transform(group1_lines)

tfidf_vectorizer_grp2 = TfidfVectorizer(**tf_vectorizer_grp2.get_params())
dtm_tfidf_grp2 = tfidf_vectorizer_grp2.fit_transform(group2_lines)

## 2.3 Compute Latent Dirichlet Allocation (LDA)
The following explanation is from [Edwin Chen's website](http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/)

LDA represents documents as mixtures of topics that spit out words with certain probabilities. It assumes that documents are produced in the following fashion: when writing each document, you:

* Decide on the number of words N the document will have (say, according to a Poisson distribution).
* Choose a topic mixture for the document (according to a Dirichlet distribution over a fixed set of K topics). For example, assuming that we have the two food and cute animal topics above, you might choose the document to consist of 1/3 food and 2/3 cute animals.
* Generate each word w_i in the document by:
    * First picking a topic (according to the multinomial distribution that you sampled above; for example, you might pick the food topic with 1/3 probability and the cute animals topic with 2/3 probability).
    * Using the topic to generate the word itself (according to the topic’s multinomial distribution). For example, if we selected the food topic, we might generate the word “broccoli” with 30% probability, “bananas” with 15% probability, and so on.
* Assuming this generative model for a collection of documents, LDA then tries to backtrack from the documents to find a set of topics that are likely to have generated the collection.

### Example

Let’s make an example. According to the above process, when generating some particular document D, you might

* Pick 5 to be the number of words in D.
* Decide that D will be 1/2 about food and 1/2 about cute animals. (Number of topics = 2)
* Pick the first word to come from the food topic, which then gives you the word “broccoli”.
* Pick the second word to come from the cute animals topic, which gives you “panda”.
* Pick the third word to come from the cute animals topic, giving you “adorable”.
* Pick the fourth word to come from the food topic, giving you “cherries”.
* Pick the fifth word to come from the food topic, giving you “eating”.
So the document generated under the LDA model will be “broccoli panda adorable cherries eating” (note that LDA is a bag-of-words model).


In [22]:
# for TF DTM
lda_tf_grp1 = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tf_grp1.fit(dtm_tf_grp1)
lda_tf_grp2 = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tf_grp2.fit(dtm_tf_grp2)
# for TFIDF DTM
lda_tfidf_grp1 = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tfidf_grp1.fit(dtm_tfidf_grp1)
lda_tfidf_grp2 = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tfidf_grp2.fit(dtm_tfidf_grp2)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

## 2.4 Visualize Free Brainstorming Topics Using Term Frequencies (unweighted)
Using PyLDAvis, visualize topics and salient terms. To focus on terms that are more salient to each topic, set $\lambda$ to 0.4.

In [6]:
# pyLDAvis.sklearn.prepare(lda_tf_grp1, dtm_tf_grp1, tf_vectorizer_grp1)

In [7]:
# pyLDAvis.sklearn.prepare(lda_tf_grp2, dtm_tf_grp2, tf_vectorizer_grp2)

## 2.4 Visualize Free Brainstorming Topics Using TF-IDF (weighted)
Using PyLDAvis, visualize topics and salient terms. To focus on terms that are more salient to each topic, set $\lambda$ to 0.4.

In [23]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1, dtm_tfidf_grp1, tfidf_vectorizer_grp1)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      23.283580        1       1  0.012815 -0.164615
2      21.174158        1       2  0.293490  0.227027
0      18.701456        1       3 -0.269877  0.059647
3      18.670648        1       4 -0.135724  0.154468
4      18.170158        1       5  0.099297 -0.276526, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
241   Default  111.000000       mean  111.000000  30.0000  30.0000
159   Default   94.000000        get   94.000000  29.0000  29.0000
457   Default   95.000000       well   95.000000  28.0000  28.0000
221   Default   76.000000     little   76.000000  27.0000  27.0000
420   Default   75.000000        toy   75.000000  26.0000  26.0000
315   Default   91.000000        put   91.000000  25.0000  25.0000
169   Default   73.000000        got   73.000000  24.0000  24.0000
434   Default   66.000000        two   66.000000  23.0000  23.0000
26    Default   54.000000       ball   54.000000  22.0000  22.0000
84    Default   64.000000       cool   64.000000  21.0000  21.0000
453   Default   58.000000        way   58.000000  20.0000  20.0000
305   Default   61.000000     pretty   61.000000  19.0000  19.0000
104   Default   51.000000       draw   51.000000  18.0000  18.0000
403   Default   50.000000       tank   50.000000  17.0000  17.0000
194   Default  111.000000       idea  111.000000  16.0000  16.0000
340   Default   57.000000        see   57.000000  15.0000  15.0000
261   Default   54.000000       need   54.000000  14.0000  14.0000
410   Default   46.000000     though   46.000000  13.0000  13.0000
448   Default   53.000000       want   53.000000  12.0000  12.0000
63    Default   44.000000        car   44.000000  11.0000  11.0000
308   Default   47.000000   probably   47.000000  10.0000  10.0000
392   Default   42.000000      stuff   42.000000   9.0000   9.0000
438   Default   42.000000        use   42.000000   8.0000   8.0000
252   Default   41.000000     motion   41.000000   7.0000   7.0000
452   Default   58.000000      water   58.000000   6.0000   6.0000
467   Default   38.000000       work   38.000000   5.0000   5.0000
136   Default   38.000000      final   38.000000   4.0000   4.0000
462   Default   37.000000       wind   37.000000   3.0000   3.0000
216   Default   64.000000        let   64.000000   2.0000   2.0000
80    Default   37.000000    concept   37.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
462    Topic5   36.685455       wind   37.443661   1.6849  -3.6877
103    Topic5   33.999990       done   34.760298   1.6833  -3.7637
336    Topic5   33.695549        say   34.456341   1.6831  -3.7727
412    Topic5   32.135621      three   32.896491   1.6820  -3.8201
431    Topic5   28.658601     trying   29.420182   1.6792  -3.9346
351    Topic5   26.364935     shoots   27.122105   1.6771  -4.0180
429    Topic5   25.197738       true   25.951646   1.6759  -4.0633
294    Topic5   25.990685       play   26.777007   1.6756  -4.0323
413    Topic5   23.927380      throw   24.686503   1.6742  -4.1150
321    Topic5   23.050676   remember   23.807756   1.6731  -4.1524
280    Topic5   22.328421       part   23.088069   1.6719  -4.1842
282    Topic5   21.903915     people   22.665474   1.6712  -4.2034
317    Topic5   18.963516     random   19.720419   1.6663  -4.3475
177    Topic5   18.977006      hafta   19.736284   1.6662  -4.3468
155    Topic5   18.056848        fun   18.814367   1.6643  -4.3965
459    Topic5   18.105787      wheel   18.865659   1.6643  -4.3938
464    Topic5   18.063246      wings   18.825146   1.6641  -4.3962
18     Topic5   17.998533       arms   18.758735   1.6640  -4.3998
32     Topic5   17.175996     battle   17.931051   1.6624  -4.4465
445    Topic5   16.615270       wait   17.370491   1.6609  -4.4797
222    Topic5   16.473697   

## 2.4 Visualize D-C Brainstorming Topics Using TF-IDF (weighted)

In [24]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2, dtm_tfidf_grp2, tfidf_vectorizer_grp2)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      22.187414        1       1  0.185932 -0.233122
3      21.377276        1       2  0.241026  0.097068
4      19.609763        1       3 -0.279546 -0.091693
0      18.420046        1       4 -0.043815  0.299592
1      18.405500        1       5 -0.103598 -0.071846, topic_info=     Category       Freq      Term      Total  loglift  logprob
term                                                           
60    Default  77.000000      cool  77.000000  30.0000  30.0000
137   Default  70.000000      kind  70.000000  29.0000  29.0000
201   Default  61.000000       put  61.000000  28.0000  28.0000
162   Default  57.000000      mean  57.000000  27.0000  27.0000
271   Default  46.000000     water  46.000000  26.0000  26.0000
130   Default  51.000000      idea  51.000000  25.0000  25.0000
115   Default  54.000000       got  54.000000  24.0000  24.0000
263   Default  37.000000       two  37.000000  23.0000  23.0000
235   Default  33.000000     start  33.000000  22.0000  22.0000
43    Default  36.000000       car  36.000000  21.0000  21.0000
70    Default  33.000000      draw  33.000000  20.0000  20.0000
145   Default  34.000000       let  34.000000  19.0000  19.0000
149   Default  30.000000    little  30.000000  18.0000  18.0000
270   Default  34.000000      want  34.000000  17.0000  17.0000
273   Default  31.000000      well  31.000000  16.0000  16.0000
109   Default  49.000000       get  49.000000  15.0000  15.0000
239   Default  28.000000     stuff  28.000000  14.0000  14.0000
77    Default  27.000000      else  27.000000  13.0000  13.0000
106   Default  28.000000      game  28.000000  12.0000  12.0000
131   Default  29.000000     ideas  29.000000  11.0000  11.0000
254   Default  42.000000       toy  42.000000  10.0000  10.0000
122   Default  26.000000       guy  26.000000   9.0000   9.0000
167   Default  28.000000    motion  28.000000   8.0000   8.0000
280   Default  23.000000      work  23.000000   7.0000   7.0000
223   Default  26.000000    shoots  26.000000   6.0000   6.0000
190   Default  26.000000      play  26.000000   5.0000   5.0000
214   Default  25.000000       see  25.000000   4.0000   4.0000
212   Default  24.000000       say  24.000000   3.0000   3.0000
119   Default  24.000000     guess  24.000000   2.0000   2.0000
9     Default  22.000000  anything  22.000000   1.0000   1.0000
...       ...        ...       ...        ...      ...      ...
3      Topic5  19.609866       air  20.363203   1.6548  -3.6032
136    Topic5  18.870506      kids  19.622028   1.6535  -3.6417
242    Topic5  17.437635      take  18.191063   1.6502  -3.7206
133    Topic5  17.317341   instead  18.071258   1.6499  -3.7276
12     Topic5  15.497449    around  16.251299   1.6450  -3.8386
284    Topic5  14.983737     write  15.734091   1.6437  -3.8723
160    Topic5  14.800451       man  15.552673   1.6429  -3.8846
151    Topic5  14.682672      look  15.435058   1.6425  -3.8926
153    Topic5  14.383044       lot  15.135888   1.6415  -3.9132
172    Topic5  13.140573      much  13.896204   1.6366  -4.0036
135    Topic5  11.523442       kid  12.277950   1.6291  -4.1349
259    Topic5  10.162922      true  10.909895   1.6216  -4.2605
169    Topic5  10.082142     mouse  10.833102   1.6207  -4.2685
229    Topic5   9.631981     sorry  10.380595   1.6177  -4.3142
203    Topic5   9.021236      real   9.775689   1.6122  -4.3797
175    Topic5   8.871557     never   9.622467   1.6113  -4.3964
164    Topic5   8.853388     might   9.604623   1.6111  -4.3985
39     Topic5   8.855915      call   9.609827   1.6108  -4.3982
200    Topic5   8.856349      push   9.610624   1.6108  -4.3981
132    Topic5   8.718909    inside   9.469994   1.6099  -4.4138
139    Topic5   8.595414     laser   9.345559   1.6088  -4.4280
148    Topic5   8.531970      line   9.284857   1.6080  -4.4355
67     Topic5   8.493127      done   9.243421   1.6079  -4.

In [25]:
filesListFreeDiv = openFiles(fileNamesFreeDivergent, textPathFree)
filesListFreeConv = openFiles(fileNamesFreeConvergent, textPathFree)
filesListDCDiv = openFiles(fileNamesDCDivergent, textPathDC)
filesListDCConv = openFiles(fileNamesDCConvergent, textPathDC)

group1_raw_div = removeSpecials(filesListFreeDiv)
group1_raw_conv = removeSpecials(filesListFreeConv)
group2_raw_div = removeSpecials(filesListDCDiv)
group2_raw_conv = removeSpecials(filesListDCConv)

group1Div = removeObservations(group1_raw_div)
group1Conv = removeObservations(group1_raw_conv)
group2Div = removeObservations(group2_raw_div)
group2Conv = removeObservations(group2_raw_conv)

group1_div_lines = group1Div.split('\n')
group1_conv_lines = group1Conv.split('\n')
group2_div_lines = group2Div.split('\n')
group2_conv_lines = group2Conv.split('\n')

print("Number of lines in the Free Group (divergent process):", len(group1_div_lines))
print("Number of lines in the Free Group (convergent process):", len(group1_conv_lines))
print("Number of lines in the D-C Group (divergent process):", len(group2_div_lines))
print("Number of lines in the D-C Group (convergent process):", len(group2_conv_lines))
print("Each line will be considered a separate document for computing the Document-Term Matrix")

Number of lines in the Free Group (divergent process): 5023
Number of lines in the Free Group (convergent process): 2619
Number of lines in the D-C Group (divergent process): 3872
Number of lines in the D-C Group (convergent process): 2769
Each line will be considered a separate document for computing the Document-Term Matrix


In [26]:
tf_vectorizer_grp1_div = CountVectorizer(strip_accents = 'unicode',
                                         analyzer = 'word',
                                         stop_words = complete_stop_words,
                                         lowercase = True,
                                         token_pattern = r'\b[a-zA-Z]{3,}\b',
                                         max_df = 0.5, 
                                         min_df = 10)

tf_vectorizer_grp1_conv = CountVectorizer(strip_accents = 'unicode',
                                          analyzer = 'word',
                                          stop_words = complete_stop_words,
                                          lowercase = True,
                                          token_pattern = r'\b[a-zA-Z]{3,}\b',
                                          max_df = 0.5, 
                                          min_df = 10)

tf_vectorizer_grp2_div = CountVectorizer(strip_accents = 'unicode',
                                         analyzer = 'word',
                                         stop_words = complete_stop_words,
                                         lowercase = True,
                                         token_pattern = r'\b[a-zA-Z]{3,}\b',
                                         max_df = 0.5, 
                                         min_df = 10)

tf_vectorizer_grp2_conv = CountVectorizer(strip_accents = 'unicode',
                                          analyzer = 'word',
                                          stop_words = complete_stop_words,
                                          lowercase = True,
                                          token_pattern = r'\b[a-zA-Z]{3,}\b',
                                          max_df = 0.5, 
                                          min_df = 10)
dtm_tf_grp1_div = tf_vectorizer_grp1.fit_transform(group1_div_lines)
print("Document-Term Matrix size for Free Group (divergent process):", dtm_tf_grp1_div.shape)
dtm_tf_grp1_conv = tf_vectorizer_grp1.fit_transform(group1_conv_lines)
print("Document-Term Matrix size for Free Group (convergent process):", dtm_tf_grp1_conv.shape)

dtm_tf_grp2_div = tf_vectorizer_grp2.fit_transform(group2_div_lines)
print("Document-Term Matrix size for Free Group (divergent process):", dtm_tf_grp2_div.shape)
dtm_tf_grp2_conv = tf_vectorizer_grp2.fit_transform(group2_conv_lines)
print("Document-Term Matrix size for Free Group (convergent process):", dtm_tf_grp2_conv.shape)

Document-Term Matrix size for Free Group (divergent process): (5023, 353)
Document-Term Matrix size for Free Group (convergent process): (2619, 159)
Document-Term Matrix size for Free Group (divergent process): (3872, 285)
Document-Term Matrix size for Free Group (convergent process): (2769, 184)


In [27]:
tfidf_vectorizer_grp1_div = TfidfVectorizer(**tf_vectorizer_grp1_div.get_params())
dtm_tfidf_grp1_div = tfidf_vectorizer_grp1_div.fit_transform(group1_div_lines)
tfidf_vectorizer_grp1_conv = TfidfVectorizer(**tf_vectorizer_grp1_conv.get_params())
dtm_tfidf_grp1_conv = tfidf_vectorizer_grp1_conv.fit_transform(group1_conv_lines)

tfidf_vectorizer_grp2_div = TfidfVectorizer(**tf_vectorizer_grp2_div.get_params())
dtm_tfidf_grp2_div = tfidf_vectorizer_grp2_div.fit_transform(group2_div_lines)
tfidf_vectorizer_grp2_conv = TfidfVectorizer(**tf_vectorizer_grp2_conv.get_params())
dtm_tfidf_grp2_conv = tfidf_vectorizer_grp2_conv.fit_transform(group2_conv_lines)

# for TFIDF DTM
lda_tfidf_grp1_div = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tfidf_grp1_div.fit(dtm_tfidf_grp1_div)
lda_tfidf_grp1_conv = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tfidf_grp1_conv.fit(dtm_tfidf_grp1_conv)

lda_tfidf_grp2_div = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tfidf_grp2_div.fit(dtm_tfidf_grp2_div)
lda_tfidf_grp2_conv = LatentDirichletAllocation(n_topics=5, random_state=0)
lda_tfidf_grp2_conv.fit(dtm_tfidf_grp2_conv)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

### Topics in The Divergent Process of the Free Brainstorming Group

In [28]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1_div, dtm_tfidf_grp1_div, tfidf_vectorizer_grp1_div)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      21.312496        1       1  0.218014 -0.048338
0      20.603645        1       2  0.148193 -0.228556
3      20.238309        1       3 -0.295996 -0.158425
1      19.611299        1       4  0.075559  0.275479
4      18.234252        1       5 -0.145770  0.159839, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
184   Default  72.000000        mean  72.000000  30.0000  30.0000
312   Default  64.000000         toy  64.000000  29.0000  29.0000
166   Default  62.000000      little  62.000000  28.0000  28.0000
340   Default  59.000000        well  59.000000  27.0000  27.0000
337   Default  58.000000       water  58.000000  26.0000  26.0000
144   Default  78.000000        idea  78.000000  25.0000  25.0000
162   Default  50.000000         let  50.000000  24.0000  24.0000
338   Default  45.000000         way  45.000000  23.0000  23.0000
124   Default  48.000000         got  48.000000  22.0000  22.0000
19    Default  43.000000        ball  43.000000  21.0000  21.0000
239   Default  63.000000         put  63.000000  20.0000  20.0000
153   Default  43.000000        kind  43.000000  19.0000  19.0000
232   Default  44.000000      pretty  44.000000  18.0000  18.0000
48    Default  39.000000         car  39.000000  17.0000  17.0000
259   Default  39.000000         see  39.000000  16.0000  16.0000
70    Default  35.000000   different  35.000000  15.0000  15.0000
17    Default  38.000000        back  38.000000  14.0000  14.0000
192   Default  36.000000      motion  36.000000  13.0000  13.0000
295   Default  34.000000       stuff  34.000000  12.0000  12.0000
277   Default  32.000000        sort  32.000000  11.0000  11.0000
313   Default  34.000000        toys  34.000000  10.0000  10.0000
63    Default  49.000000        cool  49.000000   9.0000   9.0000
325   Default  31.000000         two  31.000000   8.0000   8.0000
200   Default  31.000000        need  31.000000   7.0000   7.0000
128   Default  30.000000         gun  30.000000   6.0000   6.0000
308   Default  28.000000        time  28.000000   5.0000   5.0000
78    Default  30.000000        else  30.000000   4.0000   4.0000
117   Default  56.000000         get  56.000000   3.0000   3.0000
198   Default  28.000000        much  28.000000   2.0000   2.0000
304   Default  28.000000      though  28.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
70     Topic5  34.533271   different  35.270893   1.6807  -3.2967
277    Topic5  31.638187        sort  32.382228   1.6786  -3.3843
308    Topic5  27.444997        time  28.186293   1.6752  -3.5265
347    Topic5  22.143066        work  22.881951   1.6690  -3.7411
103    Topic5  20.623855        fish  21.357998   1.6669  -3.8122
10     Topic5  20.405406     another  21.141989   1.6664  -3.8229
140    Topic5  19.877716         hit  20.620439   1.6652  -3.8491
272    Topic5  14.899913       small  15.637452   1.6536  -4.1373
323    Topic5  14.833232        turn  15.569992   1.6534  -4.1418
189    Topic5  14.397453       might  15.134720   1.6519  -4.1716
102    Topic5  14.368207       first  15.110096   1.6515  -4.1737
283    Topic5  14.270785    spinning  15.011691   1.6513  -4.1805
33     Topic5  13.467637      bounce  14.204212   1.6486  -4.2384
180    Topic5  13.464519      marble  14.203066   1.6485  -4.2386
120    Topic5  13.356820        give  14.092441   1.6483  -4.2466
327    Topic5  13.233616  underwater  13.968496   1.6478  -4.2559
190    Topic5  13.182030     minutes  13.914757   1.6478  -4.2598
315    Topic5  12.793740       train  13.530382   1.6459  -4.2897
80     Topic5  12.647680      enough  13.385508   1.6452  -4.3012
142    Topic5  12.348920        hold  13.082690   1.6441  -4.3251
168    Topic5  12.031897        look  12.770702   1.6423  -4.3511
237    Topic5  

### Topics in The Convergent Process of the Free Brainstorming Group

In [29]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1_conv, dtm_tfidf_grp1_conv, tfidf_vectorizer_grp1_conv)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      21.900084        1       1 -0.099263  0.266009
4      20.316969        1       2  0.061237 -0.265663
2      20.141225        1       3  0.243078  0.135744
1      19.997758        1       4 -0.310111 -0.057746
0      17.643965        1       5  0.105059 -0.078344, topic_info=     Category       Freq      Term      Total  loglift  logprob
term                                                           
62    Default  43.000000      idea  43.000000  30.0000  30.0000
51    Default  45.000000       get  45.000000  29.0000  29.0000
149   Default  45.000000      well  45.000000  28.0000  28.0000
127   Default  37.000000      tank  37.000000  27.0000  27.0000
100   Default  29.000000  probably  29.000000  26.0000  26.0000
142   Default  37.000000       two  37.000000  25.0000  25.0000
87    Default  27.000000      need  27.000000  24.0000  24.0000
71    Default  27.000000    little  27.000000  23.0000  23.0000
69    Default  24.000000       let  24.000000  22.0000  22.0000
79    Default  42.000000      mean  42.000000  21.0000  21.0000
35    Default  25.000000      draw  25.000000  20.0000  20.0000
136   Default  25.000000       toy  25.000000  19.0000  19.0000
146   Default  24.000000      want  24.000000  18.0000  18.0000
45    Default  25.000000     final  25.000000  17.0000  17.0000
99    Default  25.000000    pretty  25.000000  16.0000  16.0000
11    Default  23.000000      back  23.000000  15.0000  15.0000
13    Default  20.000000      ball  20.000000  14.0000  14.0000
74    Default  22.000000     looks  22.000000  13.0000  13.0000
153   Default  21.000000      wind  21.000000  12.0000  12.0000
107   Default  22.000000       see  22.000000  11.0000  11.0000
148   Default  21.000000       way  21.000000  10.0000  10.0000
30    Default  21.000000      cool  21.000000   9.0000   9.0000
102   Default  31.000000       put  31.000000   8.0000   8.0000
77    Default  21.000000       man  21.000000   7.0000   7.0000
156   Default  18.000000      work  18.000000   6.0000   6.0000
130   Default  20.000000    though  20.000000   5.0000   5.0000
54    Default  32.000000       got  32.000000   4.0000   4.0000
119   Default  19.000000    spring  19.000000   3.0000   3.0000
125   Default  18.000000      sure  18.000000   2.0000   2.0000
6     Default  16.000000  anything  16.000000   1.0000   1.0000
...       ...        ...       ...        ...      ...      ...
62     Topic5  42.469855      idea  43.212676   1.7174  -2.1985
69     Topic5  23.354317       let  24.085698   1.7039  -2.7965
13     Topic5  20.000977      ball  20.731764   1.6989  -2.9515
156    Topic5  17.932245      work  18.660780   1.6950  -3.0607
6      Topic5  15.290724  anything  16.018286   1.6883  -3.2200
135    Topic5  14.125888       top  14.859740   1.6841  -3.2993
64     Topic5  13.678644    inside  14.408773   1.6828  -3.3314
83     Topic5  13.295211    motion  14.023157   1.6815  -3.3599
75     Topic5  13.272332       lot  14.007024   1.6809  -3.3616
21     Topic5  12.705278      call  13.431878   1.6792  -3.4052
120    Topic5  11.877057     start  12.607353   1.6751  -3.4727
66     Topic5  10.115296      kids  10.843750   1.6652  -3.6332
37     Topic5   9.900986      dude  10.634357   1.6633  -3.6546
94     Topic5   9.328296    person  10.057841   1.6595  -3.7142
124    Topic5   8.944958  supposed   9.675653   1.6563  -3.7562
111    Topic5   8.802462     shoot   9.532256   1.6551  -3.7722
141    Topic5   8.704051      turn   9.432805   1.6544  -3.7835
40     Topic5   8.601959      else   9.329445   1.6536  -3.7953
110    Topic5   8.496391      ship   9.222301   1.6528  -3.8076
10     Topic5   8.346429   awesome   9.072246   1.6514  -3.8254
133    Topic5   8.135227     throw   8.866162   1.6487  -3.8511
61     Topic5   8.060333      head   8.788868   1.6482  -3.8603
39     Topic5   7.227725    either   7.956416   1.6387  -3.

### Topics in The Divergent Process of the D-C Brainstorming Group

In [30]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2_div, dtm_tfidf_grp2_div, tfidf_vectorizer_grp2_div)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      22.187414        1       1  0.185932 -0.233122
3      21.377276        1       2  0.241026  0.097068
4      19.609763        1       3 -0.279546 -0.091693
0      18.420046        1       4 -0.043815  0.299592
1      18.405500        1       5 -0.103598 -0.071846, topic_info=     Category       Freq      Term      Total  loglift  logprob
term                                                           
60    Default  77.000000      cool  77.000000  30.0000  30.0000
137   Default  70.000000      kind  70.000000  29.0000  29.0000
201   Default  61.000000       put  61.000000  28.0000  28.0000
162   Default  57.000000      mean  57.000000  27.0000  27.0000
271   Default  46.000000     water  46.000000  26.0000  26.0000
130   Default  51.000000      idea  51.000000  25.0000  25.0000
115   Default  54.000000       got  54.000000  24.0000  24.0000
263   Default  37.000000       two  37.000000  23.0000  23.0000
235   Default  33.000000     start  33.000000  22.0000  22.0000
43    Default  36.000000       car  36.000000  21.0000  21.0000
70    Default  33.000000      draw  33.000000  20.0000  20.0000
145   Default  34.000000       let  34.000000  19.0000  19.0000
149   Default  30.000000    little  30.000000  18.0000  18.0000
270   Default  34.000000      want  34.000000  17.0000  17.0000
273   Default  31.000000      well  31.000000  16.0000  16.0000
109   Default  49.000000       get  49.000000  15.0000  15.0000
239   Default  28.000000     stuff  28.000000  14.0000  14.0000
77    Default  27.000000      else  27.000000  13.0000  13.0000
106   Default  28.000000      game  28.000000  12.0000  12.0000
131   Default  29.000000     ideas  29.000000  11.0000  11.0000
254   Default  42.000000       toy  42.000000  10.0000  10.0000
122   Default  26.000000       guy  26.000000   9.0000   9.0000
167   Default  28.000000    motion  28.000000   8.0000   8.0000
280   Default  23.000000      work  23.000000   7.0000   7.0000
223   Default  26.000000    shoots  26.000000   6.0000   6.0000
190   Default  26.000000      play  26.000000   5.0000   5.0000
214   Default  25.000000       see  25.000000   4.0000   4.0000
212   Default  24.000000       say  24.000000   3.0000   3.0000
119   Default  24.000000     guess  24.000000   2.0000   2.0000
9     Default  22.000000  anything  22.000000   1.0000   1.0000
...       ...        ...       ...        ...      ...      ...
3      Topic5  19.609866       air  20.363203   1.6548  -3.6032
136    Topic5  18.870506      kids  19.622028   1.6535  -3.6417
242    Topic5  17.437635      take  18.191063   1.6502  -3.7206
133    Topic5  17.317341   instead  18.071258   1.6499  -3.7276
12     Topic5  15.497449    around  16.251299   1.6450  -3.8386
284    Topic5  14.983737     write  15.734091   1.6437  -3.8723
160    Topic5  14.800451       man  15.552673   1.6429  -3.8846
151    Topic5  14.682672      look  15.435058   1.6425  -3.8926
153    Topic5  14.383044       lot  15.135888   1.6415  -3.9132
172    Topic5  13.140573      much  13.896204   1.6366  -4.0036
135    Topic5  11.523442       kid  12.277950   1.6291  -4.1349
259    Topic5  10.162922      true  10.909895   1.6216  -4.2605
169    Topic5  10.082142     mouse  10.833102   1.6207  -4.2685
229    Topic5   9.631981     sorry  10.380595   1.6177  -4.3142
203    Topic5   9.021236      real   9.775689   1.6122  -4.3797
175    Topic5   8.871557     never   9.622467   1.6113  -4.3964
164    Topic5   8.853388     might   9.604623   1.6111  -4.3985
39     Topic5   8.855915      call   9.609827   1.6108  -4.3982
200    Topic5   8.856349      push   9.610624   1.6108  -4.3981
132    Topic5   8.718909    inside   9.469994   1.6099  -4.4138
139    Topic5   8.595414     laser   9.345559   1.6088  -4.4280
148    Topic5   8.531970      line   9.284857   1.6080  -4.4355
67     Topic5   8.493127      done   9.243421   1.6079  -4.

### Topics in The Convergent Process of the D-C Brainstorming Group

In [31]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2_conv, dtm_tfidf_grp2_conv, tfidf_vectorizer_grp2_conv)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      22.636800        1       1  0.317661 -0.080176
1      21.066095        1       2 -0.082333  0.303393
4      19.610391        1       3 -0.239604 -0.235749
2      18.362929        1       4  0.051074 -0.055706
3      18.323784        1       5 -0.046797  0.068237, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
130   Default  46.000000        put  46.000000  30.0000  30.0000
81    Default  37.000000       idea  37.000000  29.0000  29.0000
90    Default  40.000000       kind  40.000000  28.0000  28.0000
92    Default  35.000000        let  35.000000  27.0000  27.0000
57    Default  34.000000      final  34.000000  26.0000  26.0000
35    Default  35.000000       cool  35.000000  25.0000  25.0000
14    Default  38.000000       ball  38.000000  24.0000  24.0000
73    Default  40.000000        got  40.000000  23.0000  23.0000
176   Default  35.000000       want  35.000000  22.0000  22.0000
26    Default  46.000000        car  46.000000  21.0000  21.0000
32    Default  26.000000    concept  26.000000  20.0000  20.0000
178   Default  26.000000        way  26.000000  19.0000  19.0000
38    Default  27.000000  different  27.000000  18.0000  18.0000
179   Default  26.000000       well  26.000000  17.0000  17.0000
40    Default  25.000000       draw  25.000000  16.0000  16.0000
172   Default  41.000000        two  41.000000  15.0000  15.0000
139   Default  24.000000        see  24.000000  14.0000  14.0000
159   Default  25.000000       take  25.000000  13.0000  13.0000
82    Default  21.000000      ideas  21.000000  12.0000  12.0000
160   Default  22.000000   thinking  22.000000  11.0000  11.0000
122   Default  22.000000      point  22.000000  10.0000  10.0000
112   Default  37.000000       need  37.000000   9.0000   9.0000
44    Default  21.000000   elephant  21.000000   8.0000   8.0000
181   Default  22.000000     wheels  22.000000   7.0000   7.0000
120   Default  18.000000       pick  18.000000   6.0000   6.0000
94    Default  20.000000     little  20.000000   5.0000   5.0000
182   Default  18.000000       work  18.000000   4.0000   4.0000
153   Default  20.000000      start  20.000000   3.0000   3.0000
13    Default  21.000000       back  21.000000   2.0000   2.0000
158   Default  19.000000       sure  19.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
57     Topic5  33.722598      final  34.469911   1.6751  -2.6514
32     Topic5  25.868553    concept  26.615896   1.6685  -2.9166
182    Topic5  17.760883       work  18.508847   1.6557  -3.2926
173    Topic5  15.648949        use  16.399848   1.6501  -3.4192
121    Topic5  15.558497       play  16.312663   1.6496  -3.4250
12     Topic5  15.016817    awesome  15.764012   1.6484  -3.4604
183    Topic5  14.496996      write  15.245894   1.6466  -3.4957
76     Topic5  14.058227        guy  14.808456   1.6450  -3.5264
123    Topic5  12.827079     pretty  13.580882   1.6399  -3.6181
157    Topic5  12.375470   supposed  13.124388   1.6382  -3.6539
29     Topic5  12.262052    combine  13.013147   1.6375  -3.6631
4      Topic5  12.167786    already  12.913120   1.6375  -3.6708
108    Topic5  11.258079     motion  12.005936   1.6327  -3.7485
165    Topic5  11.271314        top  12.021877   1.6325  -3.7474
100    Topic5  11.192260       made  11.939089   1.6324  -3.7544
53     Topic5  10.910898        fan  11.661408   1.6304  -3.7799
39     Topic5  10.244631       done  10.994296   1.6263  -3.8429
143    Topic5  10.143852     sheets  10.897373   1.6253  -3.8527
34     Topic5   9.665084    control  10.411427   1.6226  -3.9011
149    Topic5   9.413225      sorry  10.158683   1.6208  -3.9275
52     Topic5   9.454019    explode  10.203046   1.6207  -3.9232
136    Topic5   9.389022       said  10.137389   1.6203  -3.9301
64  